# 1. 이론적 배경

## 관련 논문

LeDell, E., & Poirier, S. (2020, July). 
H2o automl: Scalable automatic machine learning. 
In Proceedings of the AutoML Workshop at ICML (Vol. 2020).

## 탐색 공간 정의
H2O의 탐색 공간은 전처리와 모델 및 하이퍼 파라미터로 구분할 수 있으며, 실험과 경험을 통해 정의 합니다.

랜덤 포레스트의 하이퍼 파라미터는 하나로 고정되는 등 탐색 공간이 매우 작음

→ 다양한 값을 탐색하는 데서 오는 성능 향상보다는 빠르고 신뢰성 있는 모델을 학습하는 것에 초점을 뒀기 때문

## H2O의 탐색 공간으로부터 알 수 있는 점
H2O AutoML 처럼 실험과 경험으로 탐색 공간을 적절히 정의하는 것이 머신러닝 자동화 시스템 개발에서 가장 중요하다고 생각합니다.

데이터 유형에 따라 적합한 탐색 공간이 다르므로, 커스터 마이징된 머신러닝 자동화 시스템 개발이 필요함

## 최적화 알고리즘과 앙상블 모델

가장 단순한 방법인 랜덤 서치를 사용하여 모델 선택과 하이퍼 파라미터 튜닝 문제를 해결함
- 실험과 경험을 통해서 탐색 공간을 굉장히 좁게 설정했기 때문에 가능함
- XGBoost를 가장 먼저 탐색하고 다음으로 그래디언트 부스팅 머신을 탐색하는 등 탐색 순서와 각 모델을 탐색하는 시간의 비율도 정해져 있음

모델을 모두 학습한 뒤 두 개의 스태킹 앙상블 모델을 추가로 학습함
- 첫 번째 스태킹 앙상블 모델인 All Models는 학습한 전체 모델을 사용해 학습함
- 두 번째 스태킹 앙상블 모델인 Best of Family는 각 알고리즘(예: XGBoost, 심층 신경망 등)마다 최고 성능의 모델만 사용해 학습함
- H2O AutoML 개발자에 따르면, 일반적으로 이 두 앙상블 모델이 개별 모델보다 우수한 결과를 보인다고 함

# 2. 자동화 실습

## 패키지 설치
H2O AutoML은 다음과 같이 설치할 수 있습니다.

In [ ]:
# %pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

## 초기화
init 메서드를 사용하면 h2o를 초기화할 수 있으며, max_log_file_size, max_mem_size, min_mem_size 등의 인자로 분석 환경을 설정할 수 있습니다.

초기화

In [ ]:
import h2o
h2o.init()

- init 메서드를 사용하면 h2o에서 사용할 클러스터의 가동 시간, 시간대, 할당된 컴퓨팅 자원과 관련된 정보가 함께 출력됨.
- H2O AutoML의 환경과 현재 사용자 수 등에 따라 해당 정보가 바뀔 수 있습니다.

## 예제 데이터 불러오기
h2O를 사용해 학습할 예제 회귀 데이터를 불러옵니다.

예제 데이터 불러오기

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('../data/regression/wankara.csv')
train_df, test_df = train_test_split(df)

- 라인 4: 특징과 라벨을 분리하지 않은 채로 학습 데이터와 평가 데이터로 분할 했습니다. 그 이유는 H2OAutoML에서는 H2OFrame이라는 데이터프레임에 특징과 라벨이 모두 포함돼야 모델을 학습할 수 있기 때문입니다.

## H2OAtuoML 클래스
h2o에서는 H2OAtuoML이라는 클래스를 사용해 머신러닝 자동화를 수행하는 인스턴스를 만들 수 있습니다.

주요 인자
|인자|설명|
|---|---|
|max_runtime_secs|최대 학습 시간(초)|
|max_models|고려하는 모델 수|
|max_runtime_secs_per_model|모델별 최대 학습 시간(초)|
|exlucde_algos|학습하지 않을 모델 목록|

인스턴스 생성

In [ ]:
from h2o.automl import H2OAutoML
aml = H2OAutoML(max_runtime_secs = 3600)

## tarin 메서드
H2OAutoML 의 인스턴스는 train이라는 메서드를 사용해 여러 모델을 학습하고 평가합니다.

train 메서드의 주요 인자
|인자|설명|
|---|---|
|x|특징 이름 목록|
|y|라벨 이름|
|training_frame|학습 데이터 (H2O Frame)|
|leaderboard_frame|검증 데이터 (H2O Frame)|

H2O Frame으로 변환

In [ ]:
train_df = h2o.H2OFrame(train_df)
test_df = h2o.H2OFrame(test_df)

H2OAutoML의 인스턴스는 train이라는 메서드를 사용해 여러 모델을 학습하고 평가합니다.

회귀모델 학습 자동화

In [ ]:
aml.train(x = list(df.drop('y', axis = 1).columns),
    y = 'y',
    training_frame = train_df,
    leaderboard_frame = test_df)

## 모델 평가
H2O Frame인 리더보드는 leaderboard 속성을 이용해 확인할 수 있으며 head 메서드를 사용해 일부를 확인할 수 있습니다.

회귀모델 학습 결과 확인

In [ ]:
lb = aml.leaderboard
display(lb.head(rows = 10))

- BestofFamily_6 스태킹 앙상블의 성능이 가장 좋았음
- 상위 10개 모델이 모두 스태킹 앙상블 혹은 딥러닝 모델임

## 모델 활용
리더보드에서 가장 점수가 높은 모델을 이용해 라벨을 예측하려면 predict 메서드를 사용합니다. 단, 입력하는 데이터 역시 H2OFrame이어야 합니다.

회귀모델 활용

In [ ]:
aml.predict(test_df)